## Packet loss alarms

##### This lab provides list of src-dest hosts which have avarege packet loss > 0. It then collects the admin information and prepares a table with contacts.

In [1]:
import helpers as hp
import pandas as pd

from datetime import datetime, timedelta
from time import mktime

Get timestamp for the last 3 days

In [2]:
date_to = datetime.today()
date_from = date_to - timedelta(days=7)
print(date_from, date_to)
d_from = int(mktime(date_from.timetuple())*1000)
d_to = int(mktime(date_to.timetuple())*1000)
print(d_from, d_to)

2020-03-02 13:41:50.151787 2020-03-09 13:41:50.151787
1583156510000 1583761310000


Query ps_meta and extract the administrator information for each host name

In [3]:
es = hp.es

query = {
  "size" : 0,
  "query" : {
    "bool" : {
      "must" : {
        "range": {
          "timestamp": {
            "gte": d_from,
            "lte": d_to 
          }
        }
      }
    }
  },
  "aggregations" : {
    "groupby" : {
      "composite" : {
        "size" : 9999,
        "sources" : [
          {
            "host" : {
              "terms" : {
                "field" : "host.keyword",
                "missing_bucket" : True,
                "order" : "asc"
              }
            }
          },
          {
            "email" : {
              "terms" : {
                "field" : "administrator.email",
                "missing_bucket" : True
              }
            }
          },
          {
            "name" : {
              "terms" : {
                "field" : "administrator.name",
                "missing_bucket" : True
              }
            }
          },
          {
            "organization" : {
              "terms" : {
                "field" : "administrator.organization",
                "missing_bucket" : True
              }
            }
          }
        ]
      }
    }
  }
}

data = es.search("ps_meta", body=query)
ha_list = []
for item in data["aggregations"]["groupby"]["buckets"]:
    
    host = item['key']['host']
    admin = item['key']['email'].lower() if item['key']['email'] != None else ''
    name = item['key']['name']
    org = item['key']['organization']
    
    ha_list.append({'host': host, 'email': admin, 'name': name, 'organization': org})
    
admin_df = pd.DataFrame(ha_list, columns=['host', 'email', 'name', 'organization'])
admin_df.sort_values('host', ascending=True, inplace=True) 
# admin_df.to_csv('admin_info.csv', index=False)
admin_df.head()

,host,email,name,organization
1,,duncan.rand@jisc.ac.uk,Duncan Rand,None
2,,edg-site-admin@qmul.ac.uk,Terry Froy,None
3,,grid.admin@iphc.cnrs.fr,Yannick Patois,IPHC / CNRS
4,,grid.admin@lpnhe.in2p3.fr,None,None
5,,sampa-admin@if.usp.br,SAMPA Admin,Instituto de Fisica da Universidade de Sao Paulo


Get the average packet loss for each src-dest pair except the once with 0 loss

In [4]:
query = {
  "size" : 0,
  "query" : {
    "range" : {
      "timestamp" : {
        "from" : d_from,
        "to" : d_to,
      }
    }
  },
  "aggregations" : {
    "groupby" : {
      "composite" : {
        "size" : 9999,
        "sources" : [
          {
            "src_host" : {
              "terms" : {
                "field" : "src_host",
                "order" : "asc"
              }
            }
          },
          {
            "dest_host" : {
              "terms" : {
                "field" : "dest_host",
                "order" : "asc"
              }
            }
          }
        ]
      },
      "aggregations" : {
        "avg_loss" : {
          "avg" : {
            "field" : "packet_loss"
          }
        },
        "having.25544" : {
          "bucket_selector" : {
            "buckets_path" : {
              "a0" : "avg_loss"
            },
            "script" : {
              "source" : "InternalSqlScriptUtils.nullSafeFilter(InternalSqlScriptUtils.gt(params.a0,params.v0))",
              "lang" : "painless",
              "params" : {
                "v0" : 0
              }
            },
            "gap_policy" : "skip"
          }
        }
      }
    }
  }
}


data = es.search("ps_packetloss", body=query)
results = []
for item in data["aggregations"]["groupby"]["buckets"]:
    results.append({'src_host':item['key']['src_host'], 'dest_host':item['key']['dest_host'], 'avg_loss':item['avg_loss']['value']})

Fix the host names and remove those not present in ps_meta (meaning they are not part of the configuration)

In [5]:
p_data = hp.ProcessHosts(results)

Start: 13:41:53
Time elapsed = 5s
Number of unique hosts: 332


Check how much data was reduced

In [6]:
print(len(results), len(p_data))

3913 3370


Create a dataframe and explore the data in a heatmap

In [7]:
df = pd.DataFrame(p_data)
df.head()

,avg_loss,dest_host,src_host
0,0.000024,psonar4.fnal.gov,hep-ps-lt-bp.pp.rl.ac.uk
1,0.002058,ps002.gla.scotgrid.ac.uk,hep-ps-lt-bp.pp.rl.ac.uk
2,0.000113,serv04.hep.phy.cam.ac.uk,hep-ps-lt-bp.pp.rl.ac.uk
3,0.000031,psum01.aglt2.org,hep-ps-lt-bp.pp.rl.ac.uk
4,0.000194,perfsonar01.hep.wisc.edu,perfson1.zeuthen.desy.de


In [8]:
import plotly as py
import plotly.graph_objects as go

fig = go.Figure(go.Heatmap(
        x=df['src_host'],
        y=df['dest_host'],
        z=df['avg_loss'],
        hovertext=df['avg_loss'],
        colorscale='Ice_r'
    ))

fig.update_layout(
#     title='Avg Packet Loss for',
    xaxis = {
             'title': 'Sources', 
             'type': 'category'
             },
    yaxis = {
            'title': 'Destinations',
            'type': 'category',
            },
    font=dict(size=9),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')
py.offline.plot(fig)

'temp-plot.html'

Merge packet loss data with the admin info

In [9]:
sdf = pd.merge(df, admin_df, how='inner', left_on='src_host', right_on='host')
sdf.columns = ['avg_loss', 'dest_host', 'src_host', 'host', 'src_email', 'src_admin_name', 'src_organization']
sdf.drop('host', axis=1, inplace=True)
sdf.head()

,avg_loss,dest_host,src_host,src_email,src_admin_name,src_organization
0,0.000194,perfsonar01.hep.wisc.edu,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None
1,0.000094,btw-lat.t1.grid.kiae.ru,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None
2,0.000071,lcgps01.gridpp.rl.ac.uk,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None
3,0.000033,atrogr007.nipne.ro,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None
4,0.000106,btw-lat.grid.kiae.ru,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None


In [10]:

ddf = pd.merge(df, admin_df, how='inner', left_on='dest_host', right_on='host')
ddf.columns = ['avg_loss', 'dest_host', 'src_host', 'host', 'dest_email', 'dest_admin_name', 'dest_organization']
ddf.drop('host', axis=1, inplace=True)
ddf.head()

,avg_loss,dest_host,src_host,dest_email,dest_admin_name,dest_organization
0,0.000024,psonar4.fnal.gov,hep-ps-lt-bp.pp.rl.ac.uk,darryl@fnal.gov,Darryl Wohlt,Fermilab
1,0.000025,psonar4.fnal.gov,hep-ps-lt-bp.pp.rl.ac.uk,darryl@fnal.gov,Darryl Wohlt,Fermilab
2,0.000021,psonar4.fnal.gov,perfsonar2.ihep.ac.cn,darryl@fnal.gov,Darryl Wohlt,Fermilab
3,0.000048,psonar4.fnal.gov,btw-lat.t1.grid.kiae.ru,darryl@fnal.gov,Darryl Wohlt,Fermilab
4,0.000092,psonar4.fnal.gov,ccperfsonar2.in2p3.fr,darryl@fnal.gov,Darryl Wohlt,Fermilab


In [11]:
cdf = pd.merge(sdf, ddf,  how='inner', left_on=['avg_loss','src_host', 'dest_host'], right_on = ['avg_loss','src_host', 'dest_host'])

In [12]:
cdf = cdf[['avg_loss', 'src_host', 'src_email', 'src_admin_name', 'src_organization', 'dest_host', 'dest_email', 'dest_admin_name', 'dest_organization']]

In [13]:
cdf.head()

,avg_loss,src_host,src_email,src_admin_name,src_organization,dest_host,dest_email,dest_admin_name,dest_organization
0,0.000194,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None,perfsonar01.hep.wisc.edu,cmsops@hep.wisc.edu,"Ajit Mohapatra, Carl Vuosalo",None
1,0.000094,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None,btw-lat.t1.grid.kiae.ru,andreyss@grid.kiae.ru,Andrey Shitov,"National Research Centre ""Kurchatov Institute""..."
2,0.000071,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None,lcgps01.gridpp.rl.ac.uk,support@gridpp.rl.ac.uk,Brian Davies,None
3,0.000033,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None,atrogr007.nipne.ro,gabriel.stoicea@cern.ch,Gabriel Stoicea,IFIN-HH
4,0.000106,perfson1.zeuthen.desy.de,grid-admin@zeuthen.desy.de,DESY-ZN administrators,None,btw-lat.grid.kiae.ru,andreyss@grid.kiae.ru,Andrey Shitov,"National Research Centre ""Kurchatov Institute""..."
